#Import Packages

In [2]:
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as pltt


ModuleNotFoundError: No module named 'pandas'

#Data Reading and Processing

In [ ]:
disease = pd.read_csv("animal_disease_dataset.csv", index_col=0)

symptom_relate = pd.melt(adisease, 
                       id_vars=["Disease"], 
                       value_vars=["Symptom 1", "Symptom 2", "Symptom 3"],
                       var_name="Symptom_Position", 
                       value_name="Symptom")

#Chi-Square Test of Independence

In [ ]:
# Create a contingency table:
# Rows = symptoms, Columns = diseases, Values = counts
contingency_table = pd.crosstab(symptom_relate["Symptom"], symptom_relate["Disease"])

# Run the chi-square test of independence
chi2_stat, p_value, dof, expected = stats.chi2_contingency(contingency_table)

# Print the results
print("Chi-square Statistic:", round(chi2_stat, 2))
print("Degrees of Freedom:", dof)
print("P-value:", round(p_value, 5))